In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

train_set_path = "../../../datasets/train_set.csv"
test_set_path = "../../../datasets/test_set.csv"
train_set_sample = 0.3

use_val_set = False

nu = 0.3787645885136912
gamma = 0.7602707654909351

# Autoencoder Setup

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


def extract_encoded_features(X_data, autoencoder, device, batch_size=256):
    # Convert to PyTorch tensor and prepare data loader
    X_tensor = torch.FloatTensor(X_data)
    X_dataset = TensorDataset(X_tensor)
    X_loader = DataLoader(X_dataset, batch_size=batch_size)

    # Extract encoded features
    X_encoded = []
    with torch.no_grad():
        for data in X_loader:
            data_x = data[0].to(device)
            encoded = autoencoder.encode(data_x)
            X_encoded.append(encoded.cpu().numpy())

    return np.vstack(X_encoded)

In [ ]:
from autoencoder import BatchNormAutoencoder

existing_model_path = "../../../autoencoder/autoencoder.pth"

existing_model_architecture = {
    "input_dim": 15,
    "hidden_dims": [13, 11],
    "latent_dim": 9,
    "activation_type": "ReLU",
    "negative_slope": 1,
    "output_activation_type": "Sigmoid",
}

autoencoder = BatchNormAutoencoder(
    input_dim=existing_model_architecture["input_dim"],
    hidden_dims=existing_model_architecture["hidden_dims"],
    latent_dim=existing_model_architecture["latent_dim"],
    activation_type=existing_model_architecture["activation_type"],
    negative_slope=existing_model_architecture["negative_slope"],
    output_activation_type=existing_model_architecture["output_activation_type"],
).to(device)

# Load best model
checkpoint = torch.load(existing_model_path)
autoencoder.load_state_dict(checkpoint["model_state_dict"])

autoencoder.eval()

In [ ]:
def export_encoder_to_onnx(model, input_dim, file_path):
    model.eval()
    # Create dummy input tensor for ONNX export
    dummy_input = torch.randn(1, input_dim, device=device)

    # Create a wrapper class that only calls the encode method
    class EncoderOnly(torch.nn.Module):
        def __init__(self, autoencoder):
            super(EncoderOnly, self).__init__()
            self.autoencoder = autoencoder

        def forward(self, x):
            return self.autoencoder.encode(x)

    encoder_only = EncoderOnly(model)

    # Export the encoder model
    torch.onnx.export(
        encoder_only,
        dummy_input,
        file_path,
        export_params=True,
        opset_version=17,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
    )
    print(f"Encoder model exported to ONNX: {file_path}")

In [ ]:
# Export just the encoder part
import os

os.makedirs("saved_models/onnx", exist_ok=True)


encoder_onnx_path = "saved_models/onnx/autoencoder_encoder_cidds_001.onnx"


export_encoder_to_onnx(
    autoencoder, existing_model_architecture["input_dim"], encoder_onnx_path
)

In [ ]:
# Function to export model to ONNX format
def export_to_onnx(model, input_dim, file_path):
    model.eval()
    # Create dummy input tensor for ONNX export
    dummy_input = torch.randn(1, input_dim, device=device)

    # Export the model
    torch.onnx.export(
        model,  # model being run
        dummy_input,  # model input (or a tuple for multiple inputs)
        file_path,  # where to save the model
        export_params=True,  # store the trained parameter weights inside the model file
        opset_version=17,  # the ONNX version to export the model to
        do_constant_folding=True,  # whether to execute constant folding for optimization
        input_names=["input"],  # the model's input names
        output_names=["output"],  # the model's output names
        dynamic_axes={
            "input": {0: "batch_size"},  # variable length axes
            "output": {0: "batch_size"},
        },
    )
    print(f"Model exported to ONNX: {file_path}")

In [ ]:
onnx_path = "saved_models/onnx/autoencoder_cidds_001.onnx"
export_to_onnx(autoencoder, existing_model_architecture["input_dim"], onnx_path)

# Training

In [ ]:
import pandas as pd

train_dataset = pd.read_csv(train_set_path)
train_dataset = train_dataset.sample(
    frac=train_set_sample, random_state=42
).reset_index(drop=True)

print(f"train set count: {train_dataset.shape[0]:,}")
train_dataset.head(3)

In [ ]:
X_train = train_dataset.drop(columns=["attack_binary", "attack_categorical"]).values
X_train_encoded = extract_encoded_features(X_train, autoencoder, device)
y_train = train_dataset["attack_categorical"].values

print(X_train.shape, X_train_encoded.shape)

In [ ]:
from sklearn.svm import OneClassSVM
import time

ocsvm = OneClassSVM(kernel="rbf", gamma=gamma, nu=nu, verbose=True)

# Start timer
start_time = time.time()

# Train the model on the normal data sample
ocsvm.fit(X_train_encoded)

# End timer
end_time = time.time()

In [ ]:
# Calculate and print training time
training_time = end_time - start_time
print(f"Training time: {training_time:.6f} seconds")

# Testing

Preparing test set

In [ ]:
test_set = pd.read_csv(test_set_path)
print(f"test set count: {test_set.shape[0]:,}")

In [ ]:
from sklearn.model_selection import train_test_split

test_df, val_df = train_test_split(
    test_set,
    test_size=0.5,
    random_state=42,
    stratify=test_set["attack_categorical"],
)

In [ ]:
from imblearn.over_sampling import SMOTE

if use_val_set:
    test_df = val_df

In [ ]:
print("Test Set:", test_df.shape)

In [ ]:
# Export test_df to a CSV file
output_path = "testing_dataset/simulation_test_set.csv"

os.makedirs(os.path.dirname(output_path), exist_ok=True)


test_df.to_csv(output_path, index=False)



print(f"Test data exported to: {output_path}")

In [ ]:
# Splitting into X and y
X_test = test_df.drop(columns=["attack_binary", "attack_categorical"]).values
y_test = test_df["attack_binary"].values
y_test_class = test_df["attack_categorical"].values

if use_val_set:
    print("Using validation set for testing.")
    sampling_strategy = {
        "dos": 4000,
        "portScan": 4000,
        "bruteForce": 4000,
        "pingScan": 4000,
    }
    smote = SMOTE(random_state=42, k_neighbors=3, sampling_strategy=sampling_strategy)
    X_test, y_test_class = smote.fit_resample(X_test, y_test_class)
    y_test = np.where(y_test_class == "benign", 1, -1)

X_test_encoded = extract_encoded_features(X_test, autoencoder, device)

print(f"test set count: {X_test.shape[0]:,}")
print(f"unique values: {pd.Series(y_test_class).value_counts()}")
test_df.head(3)

Perform prediction

In [ ]:
# Start timer
start_time = time.time()

y_pred = ocsvm.predict(X_test_encoded)

# End timer
end_time = time.time()

In [ ]:
# Calculate and print training time
predict_time = end_time - start_time
print(f"Inference time: {predict_time:.6f} seconds")

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

# Define the input dimensions
# For your autoencoder latent space, this would be 9 based on your notebook
feature_count = X_train_encoded.shape[1]  # Which is 9 in your case

# Define the initial types for the model inputs
initial_types = [("float_input", FloatTensorType([None, feature_count]))]

# Convert the sklearn model to ONNX
onnx_model = convert_sklearn(ocsvm, initial_types=initial_types, target_opset=15)

# Save the model
base_ocsvm_onnx_path = "saved_models/onnx/base_ocsvm_cidds_001.onnx"
onnx.save_model(onnx_model, base_ocsvm_onnx_path)
print(f"OCSVM model exported to ONNX: {base_ocsvm_onnx_path}")

In [ ]:
import onnxruntime as rt
import numpy as np

# Create an ONNX Runtime session
session = rt.InferenceSession(base_ocsvm_onnx_path)

# Prepare some test data
# test_data = X_test_encoded[:5]  # Take a few samples for testing
test_data = X_test_encoded[:5]  # Take a few samples for testing
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Run inference
test_pred_onnx = session.run(
    [output_name], {input_name: X_test_encoded.astype(np.float32)}
)[0]

# If output is not -1/1, remap:
if set(np.unique(test_pred_onnx)).issubset({0, 1}):
    # Map 0 -> -1 (anomaly), 1 -> 1 (normal)
    test_pred_onnx = np.where(test_pred_onnx == 0, -1, 1)
elif test_pred_onnx.ndim > 1 and test_pred_onnx.shape[1] == 1:
    # Flatten if needed
    test_pred_onnx = test_pred_onnx.ravel()

# Compare with original model predictions
test_pred_sklearn = y_pred

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, test_pred_onnx, labels=[-1, 1])


def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(5, 4))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels
    )
    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")
    plt.title(title)
    plt.show()


print("Confusion Matrix of ONNX OCSVM Predictions")
plot_confusion_matrix(cm, ["Anomaly", "Normal"], "Confusion Matrix (Anomaly vs Normal)")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, test_pred_sklearn, labels=[-1, 1])


def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(5, 4))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels
    )
    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")
    plt.title(title)
    plt.show()


print("Confusion Matrix of SKLearn OCSVM Predictions")
plot_confusion_matrix(cm, ["Anomaly", "Normal"], "Confusion Matrix (Anomaly vs Normal)")

In [ ]:
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)

print("Classification Report ONNX:")
print(classification_report(y_test, test_pred_onnx, target_names=["Anomaly", "Normal"]))

precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

In [ ]:
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)

print("Classification Report SKLEARN:")
print(classification_report(y_test, test_pred_sklearn, target_names=["Anomaly", "Normal"]))

precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

In [ ]:
import numpy as np


def create_multiclass_cm(y_true_class, y_pred_binary):
    """
    Create a confusion matrix showing how each attack class was classified.

    For attack classes (DoS, Probe, R2L, U2R), correct detection is when y_pred = -1 (anomaly)
    For normal class, correct detection is when y_pred = 1 (normal)
    """
    classes = np.unique(y_true_class)
    cm = np.zeros((len(classes), 2))

    for i, cls in enumerate(classes):
        # Get predictions for this class
        cls_indices = y_true_class == cls
        preds = y_pred_binary[cls_indices]

        # Count correct and incorrect predictions
        if cls == "normal":
            cm[i, 0] = np.sum(preds == -1)  # incorrectly detected as anomaly
            cm[i, 1] = np.sum(preds == 1)  # correctly detected as normal
        else:
            cm[i, 0] = np.sum(preds == -1)  # correctly detected as anomaly
            cm[i, 1] = np.sum(preds == 1)  # incorrectly detected as normal

    return cm, classes


# Create and plot the multi-class confusion matrix
cm_multi, classes = create_multiclass_cm(y_test_class, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm_multi,
    annot=True,
    fmt="g",
    cmap="Blues",
    xticklabels=["Detected as Anomaly", "Detected as Normal"],
    yticklabels=classes,
)
plt.ylabel("True Attack Class")
plt.title("Confusion Matrix by Attack Class")
plt.tight_layout()
plt.show()

In [ ]:
# Calculate detection rates for each class
print("Detection rates by class:")
class_metrics = {}
for cls in np.unique(y_test_class):
    # Get indices for this class
    class_indices = y_test_class == cls

    # True values and predictions for this class
    y_true_cls = y_test[class_indices]
    y_pred_cls = y_pred[class_indices]

    # Calculate metrics
    if cls == "Normal":
        # For normal class, we want to detect 1 (normal)
        correct = np.sum((y_pred_cls == 1))
        precision = precision_score(
            y_true_cls, y_pred_cls, pos_label=1, zero_division=0
        )
        recall = recall_score(y_true_cls, y_pred_cls, pos_label=1, zero_division=0)
    else:
        # For attack classes, we want to detect -1 (anomaly)
        correct = np.sum((y_pred_cls == -1))
        precision = precision_score(
            y_true_cls, y_pred_cls, pos_label=-1, zero_division=0
        )
        recall = recall_score(y_true_cls, y_pred_cls, pos_label=-1, zero_division=0)

    total = len(y_pred_cls)
    detection_rate = correct / total
    f1 = f1_score(
        y_true_cls, y_pred_cls, pos_label=-1 if cls != "Normal" else 1, zero_division=0
    )

    class_metrics[cls] = {
        "detection_rate": detection_rate,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "count": total,
        "correctly_detected": correct,
    }

    print(f"{cls}: {detection_rate:.4f} ({correct}/{total})")